In [13]:
import os.path as osp
import os

#torch 전에 할댕해줘야함
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"

In [14]:


import torch
from tqdm import tqdm

from torch_geometric.datasets import AmazonBook, MovieLens100K
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree

In [15]:


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [16]:
# ml 1m 
path = osp.join('./', 'data', 'ML1M')
dataset = MovieLens100K(path)
dataset.process()
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['movie'].num_nodes
data = data.to_homogeneous().to(device)
# 이새끼 호모지니어스로 바꾸면 엣지 라벨 순서가 바뀌어 버림,,,,,,
data.edge_label_index = torch.stack([data.edge_label_index[1],data.edge_label_index[0]], dim = 0)
data.edge_label_index

tensor([[   5,    9,   11,  ...,  933,    9,  681],
        [1682, 1682, 1682,  ..., 2140, 2141, 2143]], device='cuda:0')

In [5]:
# 아마존
path = osp.join('./', 'data', 'Amazon')
dataset = AmazonBook(path)
data = dataset[0]
num_users, num_books = data['user'].num_nodes, data['book'].num_nodes
data = data.to_homogeneous().to(device)


In [60]:
# Use all message passing edges as training labels:
batch_size = 8192
mask = data.edge_index[0] < data.edge_index[1]
train_edge_label_index = data.edge_index[:, mask]
train_loader = torch.utils.data.DataLoader(
    range(train_edge_label_index.size(1)),
    shuffle=True,
    batch_size=batch_size,
)

model = LightGCN(
    num_nodes=data.num_nodes,
    embedding_dim=64,
    num_layers=2
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [61]:
def train():
    total_loss = total_examples = 0

    for index in train_loader:
        # Sample positive and negative labels.
        pos_edge_label_index = train_edge_label_index[:, index]
        neg_edge_label_index = torch.stack([
            pos_edge_label_index[0],
            torch.randint(num_users, num_users + num_books,
                          (index.numel(), ), device=device)
        ], dim=0)
        edge_label_index = torch.cat([
            pos_edge_label_index,
            neg_edge_label_index,
        ], dim=1)

        optimizer.zero_grad()
        pos_rank, neg_rank = model(data.edge_index, edge_label_index).chunk(2)

        loss = model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=edge_label_index.unique(),
            lambda_reg = 0.0001
        )
        loss.backward()
        optimizer.step()

        total_loss += float(loss) * pos_rank.numel()
        total_examples += pos_rank.numel()

    return total_loss / total_examples

In [62]:
@torch.no_grad()
def test(k: int):
    emb = model.get_embedding(data.edge_index)
    user_emb, book_emb = emb[:num_users], emb[num_users:]

    precision = recall = total_examples = 0
    for start in range(0, num_users, batch_size):
        end = min(start + batch_size, num_users)
        logits = user_emb[start:end] @ book_emb.t()
        # Exclude training edges:
        mask = ((train_edge_label_index[0] >= start) &
                (train_edge_label_index[0] < end))
        logits[train_edge_label_index[0, mask] - start,
                train_edge_label_index[1, mask] - num_users] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((data.edge_label_index[0] >= start) &
                (data.edge_label_index[0] < end))
        ground_truth[data.edge_label_index[0, mask] - start,
                        data.edge_label_index[1, mask] - num_users] = True
        node_count = degree(data.edge_label_index[0, mask] - start,
                                num_nodes=logits.size(0))

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [63]:
best = 0
best_model = model.get_embedding(data.edge_index).detach()
best_init = model.embedding.weight.detach()
p = 0
for epoch in tqdm(range(0, 1000)):
    loss = train()
    precision, recall = test(k=20)
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Precision@20: '
            f'{precision:.4f}, Recall@20: {recall:.4f}')
        if recall > best:
            p = 0
            best_model = model.get_embedding(data.edge_index).detach()
            best_init = model.embedding.weight.detach()
            best = recall
        else:
            p += 1
            if p > 10:
                break

  0%|          | 4/1000 [00:00<00:53, 18.70it/s]

Epoch: 000, Loss: 0.6929, Precision@20: 0.0688, Recall@20: 0.0721


  1%|▏         | 14/1000 [00:00<00:52, 18.75it/s]

Epoch: 010, Loss: 0.4782, Precision@20: 0.0753, Recall@20: 0.0891


  2%|▏         | 24/1000 [00:01<00:53, 18.17it/s]

...

Epoch: 680, Loss: 0.1559, Precision@20: 0.1580, Recall@20: 0.2015


 69%|██████▉   | 694/1000 [00:37<00:16, 18.63it/s]

Epoch: 690, Loss: 0.1567, Precision@20: 0.1572, Recall@20: 0.2012


 70%|███████   | 700/1000 [00:37<00:16, 18.45it/s]

Epoch: 700, Loss: 0.1569, Precision@20: 0.1569, Recall@20: 0.1998
